# Caribbean Coastal Shoreline Condition

This is a draft indicator for the 2023 Caribbean coastal shoreline condition.

Created by Blair Tirpak, modified from Coastal Shoreline Condition 2022, which was adapted from Shoreline Condition for the South Atlantic 2021, created by Amy Keister.
Rewritten by BT on Janurary 16, 2023. Updated on July 13, 2023 with new extent. 
Last run by Blair Tirpak on July 14, 2023
Updated Aug 30, 2023 to update colors and dataset name. 

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"C:\Data\Indicators\CARshoreline\CarShoreline.gdb"

In [3]:
# define final indicator outputs
VIPRout = r"C:\Data\Indicators\CARshoreline\CaribbeanCoastalShorelineCondition.tif"

In [4]:
# define rasters used for cell size, extent, and snapping
Extent= r"C:\Data\Indicators\VIPR_Extent_v6.tif"

In [5]:
# define inputs
PRVI = r"C:\Data\Indicators\CARshoreline\Southeast_Caribbean\Southeast_Caribbean.shp"

### Start Analysis

In [8]:
# Set the output workspace 
arcpy.env.workspace = OutWorkspace

In [9]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

C:\Data\Indicators\CARshoreline\CarShoreline.gdb


## Bring in national CUSP Shoreline data

In [10]:
# set code block for next step
codeblock = """
def Reclass(ATTRIBUTE):
    if "Natural" in ATTRIBUTE:
        return 1
    else:
        return 0 
"""

In [11]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("PRVI", "Shore", "Reclass(!ATTRIBUTE!)", "PYTHON3", codeblock, "SHORT")

<Result 'PRVI'>

In [12]:
# convert to raster using "Shore" attribute. 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Extent, snapRaster=Extent, cellSize=Extent):
    arcpy.conversion.FeatureToRaster("PRVI", "Shore", "indicator", Extent)

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to VIPR extent, clip and export to VIPR extent

In [10]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 1:
        return '1 = Natural'
    elif value == 0:
        return '0 = Armored'  
"""

In [11]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'C:\\Data\\Indicators\\CARshoreline\\CarShoreline.gdb\\indicator'>

In [12]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 1:
		return 66
	else:
		return 217
		
def Reclass2(Value):
	if Value == 1:
		return 130
	else:
		return 176
		
def Reclass3(Value):
	if Value == 1:
		return 0
	else:
		return 152
"""

In [13]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'C:\\Data\\Indicators\\CARshoreline\\CarShoreline.gdb\\indicator'>

In [14]:
# FULL EXTENT FILE 
# export as .tif with VIPR extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Extent, snapRaster=Extent, cellSize=Extent):
    arcpy.management.CopyRaster("indicator", VIPRout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [15]:
# clip to 2023 VIPR Blueprint extent
out_raster = arcpy.sa.ExtractByMask("indicator", Extent); out_raster.save("SEMask")

In [16]:
# export as .tif with VIPR extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Extent, snapRaster=Extent, cellSize=Extent):
    arcpy.management.CopyRaster("SEMask", VIPRout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

# FIN